# GaussianMixtures

Gaussian Mixture Models (GMM) é un algoritmo de clustering baseado en modelos probabilísticos que asume que os datos son xerados por unha combinación de varias distribucións gaussianas. Cada cluster no modelo é modelado por unha distribución gaussiana, e o algoritmo trata de estimar os parámetros (media, covarianza e peso) de cada distribución. GMM utiliza o algoritmo Expectation-Maximization (EM) para optimizar a asignación de puntos aos clusters e os parámetros, facendo unha aproximación iterativa. Este enfoque permite modelar clusters de diferentes formas e tamaños, o que o convirte nunha opción axeitada para clustering de datos con distribución non esférica.

## Dependencias

In [ ]:
!pip install numpy matplotlib scikit-learn seaborn pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import warnings

In [ ]:
warnings.filterwarnings("ignore")

### Datos

#### Exploración dos datos

In [ ]:
!wget http://fegalaz.usc.es/~sdocio/apau2/p1/datasets/customers.csv

In [ ]:
df = pd.read_csv(r'customers.csv')

In [ ]:
df.head()

Seleccionamos as columnas coas que imos traballar.

In [ ]:
df = df[['Annual Income (k$)', 'Spending Score (1-100)']]

In [ ]:
df.head()

Normalizamos os datos para axustar a escala.

In [ ]:
scaler = MinMaxScaler()

X = scaler.fit_transform(df)
df_scaled = pd.DataFrame(X, columns=df.columns, index=df.index)

In [ ]:
df_scaled.head()

### Clustering

In [ ]:
def get_optimal_clusters(X, max_clusters=20):
    max_score = -1
    optimal = 1

    for n_clusters in range(2, max_clusters + 1):
        clusterer = GaussianMixture(n_components=n_clusters, random_state=42)
        score = silhouette_score(X, clusterer.fit_predict(X))
        if score > max_score:
            max_score = score
            optimal = n_clusters
    return optimal

optimal = get_optimal_clusters(df_scaled)
print(f"Número de clusters: {optimal}")

In [ ]:
gmm = GaussianMixture(n_components=optimal, random_state=42)
labels = gmm.fit_predict(df_scaled)

In [ ]:
labels

In [ ]:
val, count = np.unique(labels, return_counts=True)

for v, c in sorted(zip(val, count), key=lambda x: x[1], reverse=True):
    print(f"  Cluster {v}: {c} puntos")

**Visualización**

In [ ]:
df['Cluster'] = labels

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', data=df, hue='Cluster', ax=ax, palette='Set2', edgecolors='k', s=75)
plt.show()

In [ ]:
for i in range(len(np.unique(labels))):
    p = df[labels == i]
    print(f"Cluster {i + 1}:")
    print(f"Observacións: {len(p)}")
    print(f"Ingresos anuais medios: {df[df['Cluster'] == i]['Annual Income (k$)'].mean() * 1000:.2f} USD")
    print(f"Perfil de gasto: {df[df['Cluster'] == i]['Spending Score (1-100)'].mean():.2f}")
    print("-------------------------------------------------")